In [9]:
# pip install psycopg2
# pip show psycopg2


In [10]:
# Importação de bibliotecas
import psycopg2

# Parâmetros de conexão
host = 'localhost'
port = 5432
database = 'transporte_coletivo'
user = 'postgres'
password = '123456789'

In [11]:
# Função para abrir conexão com o BD
def abre_conexao():
    conexao = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cursor = conexao.cursor()
    
    return conexao, cursor

In [12]:
# Função para fechar conexão com o BD
def fecha_conexao(conexao, cursor):
    cursor.close()
    conexao.close()

In [13]:
# Função para criar esquema
def cria_esquema(sql):
    
    conexao, cursor = abre_conexao()
    cursor.execute(sql)
    conexao.commit()
    fecha_conexao(conexao, cursor)

In [15]:
# Função para criar tabela
def cria_esquema(sql):
    
    conexao, cursor = abre_conexao()
    cursor.execute(sql)
    conexao.commit()
    fecha_conexao(conexao, cursor)

In [14]:
# Função para executar consulta
def executa_consulta(sql):
    
    conexao, cursor = abre_conexao()
    cursor.execute(sql)
    resultado = cursor.fetchall()
    fecha_conexao(conexao, cursor)
    
    return resultado

In [16]:
# # Exemplo de consulta
# consulta = "SELECT * FROM rodoviario.MUNICIPIO"
# resultados = executa_consulta(consulta)

# # Exibir os resultados
# for resultado in resultados:
#     print(resultado)
    
# # # Executar a instrução SQL para criar o esquema
# create_schema_query = f"CREATE SCHEMA oioioi;"
# cria_esquema(create_schema_query)

(1, 'Criciúma', 4.5)
(2, 'Içara', 4.0)
(3, 'Forquilhinha', 3.75)
(4, 'Araranguá', 4.2)
(5, 'Morro da Fumaça', 3.9)


DuplicateSchema: schema "oioioi" already exists
